In [1]:
# Setting up PySpark
import findspark
findspark.init()
findspark.find()
import pyspark

import os
import functools as reduce
from pyspark.context import SparkContext
from pyspark.sql import DataFrame, SQLContext, SparkSession, Window
import pyspark.sql.functions as F
from pyspark.sql.types import *

conf = pyspark.SparkConf().setAppName('NYC-Parking-Tickects-EDA').setMaster('local')
sc = pyspark.SparkContext(conf = conf)
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [2]:
# Reading the data into DataFrame
df = spark.read.csv("Parking_Violations_Issued_-_Fiscal_Year_2017.csv", inferSchema=True, header=True)

# 1. Examining the dataset

In [3]:
# Checking the schema of the dataframe
df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [4]:
# Checking the Columns
df.columns

['Summons Number',
 'Plate ID',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Issuing Agency',
 'Street Code1',
 'Street Code2',
 'Street Code3',
 'Vehicle Expiration Date',
 'Violation Location',
 'Violation Precinct',
 'Issuer Precinct',
 'Issuer Code',
 'Issuer Command',
 'Issuer Squad',
 'Violation Time',
 'Time First Observed',
 'Violation County',
 'Violation In Front Of Or Opposite',
 'House Number',
 'Street Name',
 'Intersecting Street',
 'Date First Observed',
 'Law Section',
 'Sub Division',
 'Violation Legal Code',
 'Days Parking In Effect    ',
 'From Hours In Effect',
 'To Hours In Effect',
 'Vehicle Color',
 'Unregistered Vehicle?',
 'Vehicle Year',
 'Meter Number',
 'Feet From Curb',
 'Violation Post Code',
 'Violation Description',
 'No Standing or Stopping Violation',
 'Hydrant Violation',
 'Double Parking Violation']

In [5]:
# Printing the top 5 rows
df.show(5)

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|Is

In [6]:
# Counting the records in the dataframe
df.count()

10803028

`Inference:` Total number of Tickets for 2017 are 10803028

In [7]:
# Counting the number of distinct Registratuiin States
df.select("Registration State").agg(F.countDistinct("Registration State")).show()

+-------------------------+
|count(Registration State)|
+-------------------------+
|                       67|
+-------------------------+



In [8]:
# Seeing the names of distinct States in the dataframe
df.select("Registration State").distinct().show(100)

+------------------+
|Registration State|
+------------------+
|                AZ|
|                SC|
|                NS|
|                LA|
|                MN|
|                NJ|
|                MX|
|                DC|
|                OR|
|                99|
|                NT|
|                VA|
|                RI|
|                WY|
|                KY|
|                BC|
|                NH|
|                MI|
|                GV|
|                NV|
|                QB|
|                WI|
|                ID|
|                CA|
|                CT|
|                NE|
|                MT|
|                NC|
|                VT|
|                MD|
|                DE|
|                MO|
|                IL|
|                ME|
|                MB|
|                WA|
|                ND|
|                MS|
|                AL|
|                IN|
|                OH|
|                TN|
|                IA|
|                NM|
|            

In [9]:
# Checking the frequency of Tickets from each State
df.groupBy("Registration State").agg(F.count("Summons Number").alias("Frequency")).sort(F.col("Frequency").desc()).show()

+------------------+---------+
|Registration State|Frequency|
+------------------+---------+
|                NY|  8481061|
|                NJ|   925965|
|                PA|   285419|
|                FL|   144556|
|                CT|   141088|
|                MA|    85547|
|                IN|    80749|
|                VA|    72626|
|                MD|    61800|
|                NC|    55806|
|                IL|    37329|
|                GA|    36852|
|                99|    36625|
|                TX|    36516|
|                AZ|    26426|
|                OH|    25302|
|                CA|    24260|
|                SC|    21836|
|                ME|    21574|
|                MN|    18227|
+------------------+---------+
only showing top 20 rows



In [10]:
# Replacing 99 in Registration State Column with NY since, it is the mode in the column
df = df.withColumn("Registration State", F.regexp_replace("Registration State", "99", "NY"))

In [11]:
df.groupBy("Registration State").agg(F.count("Summons Number").alias("Frequency")).sort(F.col("Frequency").desc()).show()

+------------------+---------+
|Registration State|Frequency|
+------------------+---------+
|                NY|  8517686|
|                NJ|   925965|
|                PA|   285419|
|                FL|   144556|
|                CT|   141088|
|                MA|    85547|
|                IN|    80749|
|                VA|    72626|
|                MD|    61800|
|                NC|    55806|
|                IL|    37329|
|                GA|    36852|
|                TX|    36516|
|                AZ|    26426|
|                OH|    25302|
|                CA|    24260|
|                SC|    21836|
|                ME|    21574|
|                MN|    18227|
|                OK|    18165|
+------------------+---------+
only showing top 20 rows



`Inference:` We can see that NY's Frequency has been increased from the last frequenct indicating that it has been replaced with "NY"

## Aggregation Tasks

#### 1. How often does each violation code occur? Display the frequency of the top five violation codes

In [12]:
df.groupBy("Violation Code").agg(F.count("Violation Code").alias("Frequency")).sort(F.col("Frequency").desc()).show(5)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|  1528588|
|            36|  1400614|
|            38|  1062304|
|            14|   893498|
|            20|   618593|
+--------------+---------+
only showing top 5 rows



`Inference:` 21 code has been violated more than 1.5 Million times followed by 36 which stands just above 1.4 Million

#### 2. How often does each "Vehicle Body Type" get a parking ticket? How about the "Vehicle Make"?

In [13]:
df.groupBy("Vehicle Body Type").agg(F.count("Summons Number").alias("Frequency")).sort(F.col("Frequency").desc()).show(5)

+-----------------+---------+
|Vehicle Body Type|Frequency|
+-----------------+---------+
|             SUBN|  3719802|
|             4DSD|  3082020|
|              VAN|  1411970|
|             DELV|   687330|
|              SDN|   438191|
+-----------------+---------+
only showing top 5 rows



`Inference:` "SUBN" is the Vehicle Body Type which gets the most tickets

In [14]:
# Let's check for Vehicle Maker as well
df.groupBy("Vehicle Make").agg(F.count("Summons Number").alias("Frequency")).sort(F.col("Frequency").desc()).show(5)

+------------+---------+
|Vehicle Make|Frequency|
+------------+---------+
|        FORD|  1280958|
|       TOYOT|  1211451|
|       HONDA|  1079238|
|       NISSA|   918590|
|       CHEVR|   714655|
+------------+---------+
only showing top 5 rows



`Inference:` "FORD" gets the most tickets

#### 3.1. "Violation Precinct" Ticket Frequency

In [15]:
# Precinct wise Ticket frequency
df.groupBy("Violation Precinct").agg(F.count("Summons Number").alias("Frequency")).sort(F.col("Frequency").desc()).show(5)

+------------------+---------+
|Violation Precinct|Frequency|
+------------------+---------+
|                 0|  2072400|
|                19|   535671|
|                14|   352450|
|                 1|   331810|
|                18|   306920|
+------------------+---------+
only showing top 5 rows



`Inference:` Ignoring 0 as a garbage value. 19th Precinct had the most tickets

#### 3.2. "Issuer Precinct" Ticket Frequency

In [16]:
# Issuer Precinct Tickets
df.groupBy("Issuer Precinct").agg(F.count("Summons Number").alias("Frequency")).sort(F.col("Frequency").desc()).show(5)

+---------------+---------+
|Issuer Precinct|Frequency|
+---------------+---------+
|              0|  2388479|
|             19|   521513|
|             14|   344977|
|              1|   321170|
|             18|   296553|
+---------------+---------+
only showing top 5 rows



`Inference:` Ignoring 0, most tickets are issued by 19th Precinct.

#### 4. Find the Violation Code Frequencies for three most precincts that have issued the most number of tickets.

In [17]:
# Registering dataframe as Temperory Table
df.createOrReplaceTempView("NYC_Parking")

In [18]:
# Since, we have registered the dataframe as table. We can now use SQL queries on the same 
spark.sql("""
select `Violation Code`, count(*) as frequency
from NYC_Parking
where `Issuer Precinct` in (19, 14, 1)
group by `Violation Code`
order by frequency desc
""").show()

+--------------+---------+
|Violation Code|frequency|
+--------------+---------+
|            14|   204922|
|            46|   122359|
|            38|    94937|
|            37|    88475|
|            69|    73736|
|            16|    71797|
|            21|    65026|
|            20|    59770|
|            31|    54198|
|            40|    35808|
|            19|    34445|
|            47|    31983|
|            71|    27890|
|            42|    27642|
|            84|    25299|
|            17|    22831|
|            10|    20579|
|            70|    15042|
|            82|    12551|
|             9|    10831|
+--------------+---------+
only showing top 20 rows



`Inference:` 14 and 46 are the Violation Codes that have been issued most by the top 3 Precinct Issuers

#### 5. Violation Code for Top 3 Precincts, i.e., 19, 14, 1

In [19]:
spark.sql("""
select * 
from (
    select *, dense_rank() over(partition by vp order by frequency desc) as rank
    from (
        select `Violation Precinct` as VP, `Violation Code` as VC, count(*) as frequency
        from NYC_Parking
        where `Violation Precinct` in (19, 14, 1)
        group by `Violation Precinct`, `Violation Code`
        order by frequency desc
        )
    )
where rank <=3
""").show()

+---+---+---------+----+
| VP| VC|frequency|rank|
+---+---+---------+----+
|  1| 14|    76375|   1|
|  1| 16|    39197|   2|
|  1| 20|    28768|   3|
| 19| 46|    90530|   1|
| 19| 38|    74926|   2|
| 19| 37|    73359|   3|
| 14| 14|    75850|   1|
| 14| 69|    58032|   2|
| 14| 31|    40150|   3|
+---+---+---------+----+



`Inference:` 
- 1st Precinct has Code-14 as the most violated.
- 19th Precinct has Code-46 as the most violated.
- 14th Precinct has Code-14 as the most violated.

In [20]:
# Converting Violation Time onto Suitable Format
df2 = df.withColumn("Violation Time", F.concat(F.col("Violation Time"), F.lit("M")))
df3 = df2.withColumn("TS", F.to_timestamp("Violation Time", "hhmma"))
df3.show(5)

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+-------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinc

In [21]:
# Inspecting top5 rows post time format treatment
df4 = df3.withColumn("Time", F.date_format(F.col("TS"), format="HH:mm"))
df4.show(5)

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+-------------------+-----+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation P

#### 6. Checking Seasonality in the Dataset

In [22]:
# Month-wise Ticket Frequency
spark.sql("""
select substr(`Issue Date`, 1,2) as Month, count(*) as Frequency
from NYC_Parking
group by Month
order by Frequency desc
""").show()

+-----+---------+
|Month|Frequency|
+-----+---------+
|   06|  1103400|
|   05|  1026534|
|   10|   969825|
|   03|   965247|
|   09|   961128|
|   11|   899849|
|   04|   888906|
|   01|   878580|
|   02|   827505|
|   08|   801774|
|   12|   779246|
|   07|   701034|
+-----+---------+



`Inference:` Most of the parking tickets were issued in June, i.e., 1,103,400

In [23]:
# Season wise Tickets Frequency
spark.sql("""
select (
case when Month in ("03","04","05") then "Spring"
when Month in ("06","07","08") then "Summer"
when Month in ("09","10","11") then "Autumn"
when Month in ("12","01","02") then "Winter" else Null end ) as Seasons, sum(Frequency) as Frequency
from 
    (
    select substr(`Issue Date`, 1,2) as Month, count(*) as Frequency
    from NYC_Parking
    group by Month
    order by Frequency desc
    )
group by Seasons
""").show(12)

+-------+---------+
|Seasons|Frequency|
+-------+---------+
| Spring|  2880687|
| Summer|  2606208|
| Autumn|  2830802|
| Winter|  2485331|
+-------+---------+



`Inference:` The highest number of Tickets were issued in Spring season but there is noot much of a significant difference in Frequency across Seasons

In [24]:
# Top violation Codes based on Tickets issued
spark.sql("""
select * 
from (
    select *, dense_rank() over (partition by Seasons order by Frequency desc) as Rank
    from 
    (
    select (
    case when Month in ("03","04","05") then "Spring"
    when Month in ("06","07","08") then "Summer"
    when Month in ("09","10","11") then "Autumn"
    when Month in ("12","01","02") then "Winter" else Null end ) as Seasons, VC, sum(Frequency) as Frequency
    from 
        (
        select substr(`Issue Date`, 1,2) as Month, `Violation Code` as VC, count(*) as Frequency
        from NYC_Parking
        group by Month, `Violation Code`
        order by Frequency desc
        )
    group by Seasons, VC
    )
)
where Rank <= 3
""").show(20)

+-------+---+---------+----+
|Seasons| VC|Frequency|Rank|
+-------+---+---------+----+
| Spring| 21|   402807|   1|
| Spring| 36|   344834|   2|
| Spring| 38|   271192|   3|
| Summer| 21|   405961|   1|
| Summer| 38|   247561|   2|
| Summer| 36|   240396|   3|
| Autumn| 36|   456046|   1|
| Autumn| 21|   357479|   2|
| Autumn| 38|   283828|   3|
| Winter| 21|   362341|   1|
| Winter| 36|   359338|   2|
| Winter| 38|   259723|   3|
+-------+---+---------+----+



`Inference:` Apparently, top violation codes for all seasons are 21, 36, 38.

#### 7. Revenue Collection from Tickets

In [25]:
# SQL Query for Violation Code Tickets frequency
spark.sql("""
select `Violation Code`, count(*) as Frequency
from NYC_Parking
group by `Violation Code`
order by Frequency desc
""").show(10)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|  1528588|
|            36|  1400614|
|            38|  1062304|
|            14|   893498|
|            20|   618593|
|            46|   600012|
|            37|   596769|
|            71|   521308|
|            40|   519615|
|             7|   516395|
+--------------+---------+
only showing top 10 rows



`Inference:` Major Violation were
- "Failure to Display Muni Rec"
- "Speed Violation in School Zones"
- "Parking in No-Parking Zones"